In [ ]:
%pip install -q tiktoken
%pip install -q cohere
%pip install -q openai
%pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.8/207.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/361.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6

In [ ]:
import os
import pandas as pd
from openai import OpenAI
import gradio as gr
import time
import json


## Set the API key and model name
MODEL="gpt-4o-mini-2024-07-18"
client = OpenAI(api_key="sk-proj-upOVy97VapEQl5kOiTQIT3BlbkFJlKBrtAnjDU5YMvWrqMzd")

In [ ]:
# 데이터셋 만들기
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')
df = pd.read_excel('/content/drive/MyDrive/kdt_240424/m9_openai/data/code_ft.xlsx', engine='openpyxl')
df.head()

In [ ]:
# 학습용 평가용 데이터 분리
import pandas as pd
from sklearn.model_selection import train_test_split
file_path = '/content/drive/MyDrive/kdt_240424/m9_openai/data/code_ft.xlsx'  # Replace with your file path
data = pd.read_excel(file_path)
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 파일 경로
file_path = '/content/drive/MyDrive/kdt_240424/m9_openai/data/code_ft.xlsx'  # 실제 경로로 변경하세요
data = pd.read_excel(file_path)

# Train-test split
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# System message
system_msg = """코딩튜터는 고등학생 정보과목에서 알고리즘과 프로그래밍을 학생들이 실습할 수 있게 도와주는 튜터봇입니다. 학생이 선택한 섹션에 대해 충분한 학습을 할 수 있도록 각 섹션별 학습 항목에 대해서 하나씩 개념을 명확하게 설명하고 연습문제를 2, 3개를 제공해서 각 개념을 응용해서 이해할 수 있는 기회를 제공합니다.

    **각 섹션별 학습내용과 목표는 다음과 같습니다:**

    1. 파이썬 기초
      변수와 자료형 이해: 파이썬에서 변수와 다양한 자료형(정수, 실수, 문자열 등)을 이해하고 사용할 수 있다.
      기본 연산 수행: 기본 산술 연산(덧셈, 뺄셈, 곱셈, 나눗셈)과 논리 연산을 이해하고 활용할 수 있다.
      입출력 처리: 표준 입력과 출력 함수를 사용하여 사용자로부터 데이터를 입력받고, 결과를 출력할 수 있다.
      조건문 이해와 활용: if, elif, else 조건문을 사용하여 프로그램의 흐름을 제어할 수 있다.
      반복문 이해와 활용: for, while 반복문을 사용하여 반복적인 작업을 수행할 수 있다.
      기본 프로그램 작성: 위 개념들을 종합적으로 활용하여 간단한 파이썬 프로그램을 작성할 수 있다.
    2. 파이썬 응용
      리스트 활용: 리스트 자료형을 이해하고, 리스트의 생성, 수정, 삭제, 접근 등의 작업을 수행할 수 있다.
      조건문과 반복문 응용: 조건문과 반복문을 복합적으로 사용하여 다양한 문제를 해결할 수 있다.
      문자열 처리: 문자열 자료형의 다양한 메서드를 사용하여 문자열을 처리하고 조작할 수 있다.
      함수 작성 및 활용: 파이썬 함수의 개념을 이해하고, 매개변수와 반환값을 사용하여 재사용 가능한 코드를 작성할 수 있다.
      딕셔너리 활용: 딕셔너리 자료형을 이해하고, 키-값 쌍을 사용하여 데이터를 저장하고 조작할 수 있다.
      문제 정의와 해결: 리스트, 조건문, 반복문, 문자열 처리, 함수, 딕셔너리를 활용하여 실제 문제를 정의하고 해결할 수 있는 프로그램을 작성할 수 있다.


    **튜터링 프로세스**:

    1. 학생들을 환영합니다.
    2. 학생들이 학습할 섹션을 선택하게 합니다.
    3. 각 섹션에 포함되는 학습할 내용들에 대해서 하나씩 개념을 이해하고 문제를 풀 수 있도록 합니다. 문제는 최소한 2개 이상 제시합니다.

    4. 각 문제에 대한 학생의 답안을 신중하게 확인하고 학생들의 이해도를 향상시킬 수 있도록 적합한 설명을 추가합니다.
    5. 학습자의 이해 정도에 따라서 문제를 단계 별로 해결할 수 있도록 설명한다.
    6. 초보자가 입력한 코드에 오류가 있는지 주의 깊게 확인하고 오류를 해결할 수 있는 피드백을 제공합니다.
    7. 학습자의 이해 정도를 파악해서 추가적인 학습을 할 지 여부를 판단한 후 다음 주제로 넘어갈 지를 결정합니다.

    8. 다음 세션으로 넘어가기 전에 학습자가 각 세션에 대한 내용을 충분히 이해했는 지 확인한 후 진행합니다.
    9. 대화는 간결하게 유지하며, 최대 30자 이내로 답변합니다.

    **학생과의 상호작용**:
   튜터봇은 학생들이 자기주도 학습을 할 수 있도록 개인화 된 학습을 제공하고 모든 질의 응답을 한국어로 진행합니다.
"""

# Function to convert DataFrame to formatted JSON
def convert_to_json(df):
    formatted_data = []
    temp_message = [{"role": "system", "content": system_msg}]
    for i, row in df.iterrows():
        # Check if content exists and is a valid string
        if pd.notna(row['content']) and isinstance(row['content'], str):
            message = {"role": row['role'], "content": row['content']}
            if row['role'] == 'assistant' and pd.notna(row.get('weight')):
                message["weight"] = int(row['weight'])
            temp_message.append(message)
            if row['role'] == 'assistant':
                formatted_data.append({"messages": temp_message})
                temp_message = [{"role": "system", "content": system_msg}]
        else:
            print(f"Skipping row {i} due to missing or invalid content.")
    return formatted_data

# Convert the DataFrame to JSON format
train_dataset_formatted = convert_to_json(train_data)
val_dataset_formatted = convert_to_json(val_data)

# Print formatted JSON
import json
formatted_json = json.dumps(train_dataset_formatted, indent=2, ensure_ascii=False)
print(formatted_json)


In [ ]:
import json
ds_sample = train_dataset_formatted
with open('/content/drive/MyDrive/kdt_240424/m9_openai/data/code_ft_train.jsonl', 'w') as f:
  for line in ds_sample:
    json.dump(line, f)
    f.write('\n')

In [ ]:
import json
ds_sample = val_dataset_formatted
with open('/content/drive/MyDrive/kdt_240424/m9_openai/data/code_ft_valid.jsonl', 'w') as f:
  for line in ds_sample:
    json.dump(line, f)
    f.write('\n')

In [ ]:
import json
import tiktoken
import numpy as np
from collections import defaultdict

data_path = '/content/drive/MyDrive/kdt_240424/m9_openai/data/code_ft_train.jsonl'
# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    train_dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("First 3 examples:")
for i in range(3):
    print(f"Example {i+1}:")
    for message in train_dataset[i]["messages"]:
        print(message)
    print("\n")  # Add a newline for better readability between examples

In [ ]:
import json
import tiktoken
import numpy as np
from collections import defaultdict

data_path = '/content/drive/MyDrive/kdt_240424/m9_openai/data/code_ft_valid.jsonl'
# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    val_dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(val_dataset))
print("First example:")
for message in val_dataset[0]["messages"]:
    print(message)

데이터 오류 확인

In [ ]:
# Format error checks
format_errors = defaultdict(int)

for ex in train_dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

In [ ]:
# Format error checks
format_errors = defaultdict(int)

for ex in val_dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

fine tuning

In [ ]:
# 데이터 업로드
# JSON Lines 파일을 모델 학습에 사용하기 위해 업로드
train = client.files.create(
  file=open("/content/drive/MyDrive/kdt_240424/m9_openai/data/code_ft_train.jsonl", "rb"),
  purpose='fine-tune'
)
train

In [ ]:
valid = client.files.create(
  file=open("/content/drive/MyDrive/kdt_240424/m9_openai/data/code_ft_valid.jsonl", "rb"),
  purpose='fine-tune'
)
valid

파인튜닝

In [ ]:
client.fine_tuning.jobs.create(
  training_file='file-ymQn1PrbhGl7KLwndLNgFHMh',
  validation_file='file-dWhHK73TB1lWH6iLwg1P6vVj',
  model=MODEL,
  hyperparameters={
    "n_epochs":3
  }
)

In [ ]:
# 작업상태 확인
response = client.fine_tuning.jobs.retrieve("ftjob-bwhRTrexajY7l2yhCdZNzu5H")

# 작업 상태 확인
status = response.status

if status == "succeeded":
    print("작업이 완료되었습니다.")
    # 작업이 완료되면 결과를 가져와서 사용할 수 있습니다.
    # 예를 들어, 모델을 사용하여 텍스트 생성 또는 다른 작업을 수행할 수 있습니다.
else:
    print(f"작업 상태: {status}")

total_tokens = response.trained_tokens
print(f"사용된 총 토큰 수: {total_tokens}")

작업이 완료되었습니다.
사용된 총 토큰 수: 31776


In [ ]:
from openai import OpenAI
import gradio as gr
import subprocess

client = OpenAI(api_key="sk-proj-upOVy97VapEQl5kOiTQIT3BlbkFJlKBrtAnjDU5YMvWrqMzd")

def code_print(code, user_input):
    modified_code = code.replace('input("이름을 입력하세요: ")', f'"{user_input}"')
    result = subprocess.run(["python", "-c", modified_code], capture_output=True, text=True)
    return result.stdout or result.stderr

def answer(state, state_chatbot, text):
    if text == "":
        return state, state_chatbot, state_chatbot

    if '```' in text:
        code_parts = text.split('```')
        code = code_parts[1].strip()
        input_value = code_parts[2].strip() if len(code_parts) > 2 else ""

        output = code_print(code, input_value)

        new_state = [{'role': 'user', 'content': text},
                     {'role': 'assistant', 'content': output}]
        state = state + new_state
        state_chatbot = state_chatbot + [(text, output)]
        return state, state_chatbot, state_chatbot

    messages = state + [{'role': 'user', 'content': text}]

    res = client.chat.completions.create(
        model="ft:gpt-4o-mini-2024-07-18:bnvs::9vhRlPzZ",
        messages=messages
    )

    msg = res.choices[0].message.content

    new_state = [{'role': 'user', 'content': text},
                 {'role': 'assistant', 'content': msg}]

    state = state + new_state
    state_chatbot = state_chatbot + [(text, msg)]

    return state, state_chatbot, state_chatbot

def start_def(section):
    if not section:
        return [], [], [("섹션을 선택해주세요!", "")]
    else:
        text = f"{section} 섹션을 선택하셨습니다. 수업을 시작하겠습니다."

    system_msg = """코딩튜터는 고등학생 정보과목에서 알고리즘과 프로그래밍을 학생들이 실습할 수 있게 도와주는 튜터봇입니다. 학생이 선택한 섹션에 대해 충분한 교육을 할 수 있도록 학습할 내용을 선별해서 하나씩 전부 개념을 설명하고 연습문제를 2, 3개를 제공해서 연습할 수 있게 합니다.

    **각 섹션별 학습내용과 목표는 다음과 같습니다:**

    1. 파이썬 기초
      변수와 자료형 이해: 파이썬에서 변수와 다양한 자료형(정수, 실수, 문자열 등)을 이해하고 사용할 수 있다.
      기본 연산 수행: 기본 산술 연산(덧셈, 뺄셈, 곱셈, 나눗셈)과 논리 연산을 이해하고 활용할 수 있다.
      입출력 처리: 표준 입력과 출력 함수를 사용하여 사용자로부터 데이터를 입력받고, 결과를 출력할 수 있다.
      조건문 이해와 활용: if, elif, else 조건문을 사용하여 프로그램의 흐름을 제어할 수 있다.
      반복문 이해와 활용: for, while 반복문을 사용하여 반복적인 작업을 수행할 수 있다.
      기본 프로그램 작성: 위 개념들을 종합적으로 활용하여 간단한 파이썬 프로그램을 작성할 수 있다.
    2. 파이썬 응용
      리스트 활용: 리스트 자료형을 이해하고, 리스트의 생성, 수정, 삭제, 접근 등의 작업을 수행할 수 있다.
      조건문과 반복문 응용: 조건문과 반복문을 복합적으로 사용하여 다양한 문제를 해결할 수 있다.
      문자열 처리: 문자열 자료형의 다양한 메서드를 사용하여 문자열을 처리하고 조작할 수 있다.
      함수 작성 및 활용: 파이썬 함수의 개념을 이해하고, 매개변수와 반환값을 사용하여 재사용 가능한 코드를 작성할 수 있다.
      딕셔너리 활용: 딕셔너리 자료형을 이해하고, 키-값 쌍을 사용하여 데이터를 저장하고 조작할 수 있다.
      문제 정의와 해결: 리스트, 조건문, 반복문, 문자열 처리, 함수, 딕셔너리를 활용하여 실제 문제를 정의하고 해결할 수 있는 프로그램을 작성할 수 있다.


    **튜터링 프로세스**:

    1. 학생들을 환영합니다.
    2. 학생들이 학습할 섹션을 선택하게 합니다.
    3. 각 섹션에 포함되는 학습할 내용들을 하나씩 개념을 이해하고 문제를 풀 수 있도록 합니다.
    4. 학생의 답안을 신중하게 확인하고 올바르게 수정합니다.
    5. 학습자의 이해 정도에 따라서 문제를 단계 별로 해결할 수 있도록 설명한다.
    6. 초보자가 입력한 코드에 오류가 있는지 주의 깊게 확인하고 적절한 피드백을 제공합니다.
    7. 학습자의 이해 정도를 파악해서 추가적인 학습을 할 지 다음 주제로 넘어갈 지를 결정합니다.
    8. 대화는 간결하게 유지하며, 최대 30자 이내로 답변합니다.

    **학생과의 상호작용**:
   튜터봇은 학생들이 자기주도 학습을 할 수 있도록 개인화 된 학습을 제공하고 모든 질의 응답을 한국어로 진행합니다.
"""

    message = [{'role': 'system', 'content': system_msg},
               {'role': 'user', 'content': text}]

    res = client.chat.completions.create(
        model="ft:gpt-4o-mini-2024-07-18:bnvs::9vhRlPzZ",
        messages=message,
        temperature=0.5,
    )

    msg = res.choices[0].message.content
    message.append({'role': 'assistant', 'content': msg})
    state_chatbot = [(text, msg)]
    return message, state_chatbot, state_chatbot

with gr.Blocks(theme='JohnSmith9982/small_and_pretty') as demo:
    state = gr.State([])
    state_chatbot = gr.State([])

    with gr.Row():
        gr.HTML("""<div style="text-align: center; max-width: 550px; margin: 0 auto;">
            <div>
              <p style="font-size: 40px; font-weight: bold;">Play CodeSync</p>
            </div>
            </div>""")

    with gr.Row():
        with gr.Column(scale=6):
            md = gr.Dropdown(["파이썬 기초", "파이썬 응용"], type="value", label="섹션 선택", info="원하는 섹션을 골라주세요!")
        with gr.Column(scale=4):
            gr.HTML("""<div style="text-align: center;">
                <div>
                  <p style="font-size: 15px; font-weight: bold;">학습 시작하기</p>
                </div>
              </div>""")
            start_btn = gr.Button("시작하기")
        with gr.Column(scale=1):
            gr.HTML("""<div style="text-align: center;">
              <div>
                <p style="font-size: 15px; font-weight: bold;">다크모드 전환</p>
              </div>
            </div>""")
            toggle_dark = gr.Button(value="Toggle Dark")
            toggle_dark.click(None,
                js="""() => {
                  document.body.classList.toggle('dark');
                  document.querySelector('gradio-container').style.backgroundColor = 'var(--color-background-primary)'
                  }
                """
            )

    with gr.Row():
        with gr.Column(scale=1):
            chatbot = gr.Chatbot(elem_id='메세지창', height=650)
        with gr.Column(scale=1):
            with gr.Row():
                txt = gr.Textbox(show_label=False, placeholder='메세지를 입력해주세요!')
            with gr.Row():
                txt_btn = gr.Button("메세지 보내기")
            with gr.Row():
                txt_input = gr.Textbox(show_label=False, placeholder='코드에서 사용할 입력값을 입력하세요')
                code_btn = gr.Button("코드 실행")
            with gr.Row():
                txt_result = gr.Textbox(label='코드실행결과')

    txt_btn.click(answer, [state, state_chatbot, txt], [state, state_chatbot, chatbot])
    txt_btn.click(lambda: '', None, txt)
    txt.submit(answer, [state, state_chatbot, txt], [state, state_chatbot, chatbot])
    txt.submit(lambda: '', None, txt)

    def handle_code_execution(txt, txt_input, state, state_chatbot):
        state_list = list(state)
        state_chatbot_list = list(state_chatbot)
        new_state, new_state_chatbot, chatbot_output = answer(state_list, state_chatbot_list, f"```{txt}```{txt_input}")
        return new_state, new_state_chatbot, chatbot_output, chatbot_output[-1][1]

    code_btn.click(handle_code_execution, inputs=[txt, txt_input, state, state_chatbot], outputs=[state, state_chatbot, chatbot, txt_result])
    start_btn.click(start_def, inputs=[md], outputs=[state, state_chatbot, chatbot])

demo.launch(debug=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


themes/theme_schema@1.0.0.json:   0%|          | 0.00/13.0k [00:00<?, ?B/s]

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://61003e129837745064.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
